# ナイーブベイズテキスト分類

今回使用するデータセットはscikit-learnのニュースデータセット

In [91]:
#必要なライブラリをインポート
!pip install ipykernel
!pip install mecab-python-windows
!pip install natto-py
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import  fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

In [92]:
#データの取得
data = fetch_20newsgroups()

In [93]:
#ニュースの分類を確認
print(data.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [94]:
#学習データとテストデータを準備
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

In [95]:
#データ数の確認
print("Train Data Count =", len(train.data))
print("Test Data Count =", len(test.data))

Train Data Count = 11314
Test Data Count = 7532


In [96]:
#データ内容を確認
print (type(train.data))
print(type(train.data[1]))
print(train.data[1])

<class 'list'>
<class 'str'>
From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



In [97]:
print(train.target_names[train.target[1]])
print(train.target[1])

comp.sys.mac.hardware
4


上の記事はcomp.sys.mac.hardwareというニュースグループの4番目の記事であることが分かる

In [98]:
#文書をTf-Idfベクトルに変換する
from natto import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [99]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.6, max_features=200000, min_df=0.01, 
                                   tokenizer=lambda x: x,
                                   preprocessor=lambda x: x,
                                   use_idf=True, ngram_range=(1,3))

In [100]:
def tokenize(text):
    tokens = []
    with MeCab('-F%f[0],%f[6]') as nm:
        for n in nm.parse(text, as_nodes=True):
            # ignore any end-of-sentence nodes
            if not n.is_eos() and n.is_nor():
                klass, word = n.feature.split(',', 1)
                if klass in ['名詞', '形容詞', '形容動詞', '動詞']:
                    tokens.append(word)
    return tokens

In [101]:
#パイプラインを使用
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [102]:
#学習を実行
model.fit(train.data, train.target)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth...   vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [103]:
#作成したモデルから評価を実行
print('Train accuracy = %.3f' % model.score(train.data, train.target))
print(' Test accuracy = %.3f' % model.score(test.data, test.target))

Train accuracy = 0.933
 Test accuracy = 0.774


In [104]:
#任意のテキストデータを入力しグループの予測を行う
def predicted_group(s, train=train, model=model):
    pred = model.predict([s])
    return train.target_names[pred[0]]

In [105]:
predicted_group('the test most likely had some nuclear dimension.')

'sci.space'

上の例文は核実験についてのニュースから引用したものであるので分類としては誤りであると考えられる。

In [106]:
predicted_group('internet connections so fast they will support an entirely new way of life.')

'soc.religion.christian'

こちらの分類も、インターネット回線について扱っているニュースからの1文を宗教関連に分類しているため誤分類。

In [107]:
predicted_group('The 13 extra-base hits set a franchise record.')

'rec.sport.baseball'

このニュース記事はメジャーリーグのニュース記事から引用したものであるので正しく分類されている。

・考察
　分類予測においては、3つ目の野球のニュースのように"extre base hits"などの特定しやすいワードが入っていると分類が容易であるが、1つ目、2つ目のように一般的な文章を分類する場合はまだ精度に問題があるように見られる。また、訓練データとテストデータの評価に差が出ていることや実際の分類予測の実践でも誤分類が多く見られたことから過学習が起こっている可能性も考えられる。過学習への対策としては、正則化を行うこと、訓練データを増やすことなどが挙げられる。
 また、今回はデータの前処理とモデルの生成をscikit-learnのpipelineを用いて行った。そのため、今回のデータ分類のために前処理・推論などのオブジェクトを自作しpipelineに組み込むことができればより高い精度での分類が可能になると予測する。